In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re

from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import KNNImputer

In [2]:
df_infobase = pd.read_csv("../../data/raw/info_base_games.csv",low_memory=False)


In [3]:
#df_infobase.head(10)
#df_infobase.tail(10)
#df_infobase.dtypes
#df_infobase.shape 
#df_infobase.info()
df_infobase.describe()


,appid,name,metacritic,steam_achievements,steam_trading_cards,workshop_support,genres,achievements_total,release_date,supported_platforms
count,99167,99149,3019,99167,99167,99167,94389,38115,98861,99167
unique,95724,95004,127,2,2,2,2699,571,4788,7
top,3304770,Delirium,80,False,False,False,"Action, Adventure, Indie",10.0,Coming soon,['windows']
freq,2,5,81,50128,82285,86138,4971,937,12109,69850


In [4]:
print(((df_infobase.isnull().sum())/df_infobase.shape[0]) * 100)

appid                   0.000000
name                    0.018151
metacritic             96.955640
steam_achievements      0.000000
steam_trading_cards     0.000000
workshop_support        0.000000
genres                  4.818135
achievements_total     61.564835
release_date            0.308570
supported_platforms     0.000000
dtype: float64


In [5]:
# FOR preprocessing release date 
#df_infobase['release_date'].isnull().sum()
#df_infobase[df_infobase['release_date'].isnull()]
#df_infobase['release_date'] = pd.to_datetime(df_infobase['release_date'])
#df_infobase['release_date']
#df_infobase['release_date'] = df_infobase['release_date'].apply(lambda x: pd.to_datetime(x, infer_datetime_format=True).year)
# To be announced , Coming soon , year , dd-mm-yy , Q1 yyyy , dd-Mon

In [6]:
print("Before Preprocessing: ",df_infobase['release_date'].isnull().sum())

def preprocess_release_date(x):
    x = str(x)
    if re.search(r'(\d{1,2}) .*? ([A-Za-z]{3}) .*? (\d{4} | \d{2})', x):
        day, month, year = re.search(r'(\d{1,2}) .*? ([A-Za-z]{3}) .*? (\d{4} | \d{2})', x).groups()
        return f"{day} {month} {year}"
    elif re.search(r'([A-Za-z]{3}) .*? (\d{2}|\d{4})', x):
        month, year = re.search(r'([A-Za-z]{3}) .*? (\d{2}|\d{4})', x).groups()
        return f"1 {month} {year}"
    elif re.search(r'\b\d{1,2}-[A-Za-z]{3}-\d{2}\b', x):
        return x.replace('-', '')
    elif re.search(r'\b\d{1,2}-[A-Za-z]{3}\b', x):
        return np.nan
    elif re.search(r'\b[\d]{1,2} [A-Za-z]{3}, [\d]{4}\b', x):
        return x.replace(',', '')
    elif re.search(r'\bQ[1-4] \d{4}\b', x):
        q, y = x.split()
        quarter_map = {
            'Q1': '01 Jan',
            'Q2': '01 Apr',
            'Q3': '01 Jul',
            'Q4': '01 Oct',
        }
        return f"{quarter_map[q]} {y}"
    elif re.search(r'\b[\d]{4}\b', x):
        return "1 JAN " + str(x)
    elif x.lower() == 'to be announced':
        return np.nan
    elif x.lower() == 'coming soon':
        return np.nan
    elif x == '':
        return np.nan
    else:
        return np.nan
        #print(x)

# for each value in column release date we will apply this function to it 
df_infobase['release_date'] = df_infobase['release_date'].apply(preprocess_release_date)

#print("After Preprocessing: ",df_infobase['release_date'].isnull().sum())
#print(df_infobase.shape[0])
#df_infobase['release_date'].sample(100)


Before Preprocessing:  306


In [7]:
#df_infobase['release_date'] = pd.to_datetime(df_infobase['release_date'], format='%d %b %Y', errors='coerce')
#df_infobase['release_date'] = df_infobase['release_date'].apply(lambda x: pd.to_datetime(x, infer_datetime_format=True).year)


df_infobase['release_date'] = pd.to_datetime(df_infobase['release_date'], errors='coerce')
df_infobase['is_release_date_known'] = df_infobase['release_date'].notna().astype(int)
df_infobase['year'] = df_infobase['release_date'].dt.year.fillna(0).astype(int)
fraction_of_year = np.where(df_infobase['is_release_date_known'], (df_infobase['release_date'].dt.dayofyear - 1) / 365, 0)
df_infobase['fraction_sin'] = np.sin(2 * np.pi * fraction_of_year)
df_infobase['fraction_cos'] = np.cos(2 * np.pi * fraction_of_year)
df_infobase.drop('release_date',axis=1,inplace=True)


C:\Users\DELL\AppData\Local\Temp\ipykernel_19476\2642697024.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_infobase['release_date'] = pd.to_datetime(df_infobase['release_date'], errors='coerce')


In [8]:
#df_infobase['release_date'].isnull().sum()
#df_infobase['release_date'].describe().T
#df_infobase['release_date'].info()
#df_infobase['release_date'].head(1000)
df_infobase
#df_infobase.head(100)

,appid,name,metacritic,steam_achievements,steam_trading_cards,workshop_support,genres,achievements_total,supported_platforms,is_release_date_known,year,fraction_sin,fraction_cos
0,2574000,Femboy Burgers,NaN,True,True,True,"Casual, Indie",NaN,"['windows', 'mac', 'linux']",1,2020,-0.999991,0.004304
1,2574120,PPA Pickleball Tour 2025,NaN,True,True,True,"Indie, Simulation, Sports",18,"['windows', 'mac', 'linux']",1,2020,0.008607,-0.999963
2,2573200,Squeaky Squad,NaN,True,True,True,"Action, Adventure, Indie",27,"['windows', 'mac', 'linux']",1,2020,0.858764,0.512371
3,2573440,Paradox Metal,NaN,True,True,True,"Action, Early Access",NaN,"['windows', 'mac', 'linux']",0,0,0.000000,1.000000
4,2569520,Naturpark Lillebælt VR,NaN,True,True,True,"Action, Adventure",NaN,"['windows', 'mac', 'linux']",1,2020,-0.871706,-0.490029
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99162,1548850,Six Days in Fallujah,NaN,True,False,False,"Action, Early Access",34.0,['windows'],1,2020,0.501242,-0.865307
99163,2478130,PROJECT SURVIVAL #Working title,NaN,False,False,False,"Action, Adventure, RPG",NaN,['windows'],0,0,0.000000,1.000000
99164,3272980,Siren's Well,NaN,True,False,False,"Action, Adventure",NaN,"['windows', 'linux']",0,0,0.000000,1.000000
99165,2054150,Tower Defender VR: Last Adventure,NaN,False,False,False,"Casual, Indie, RPG, Strategy",NaN,['windows'],1,2020,0.008607,-0.999963


I have mutliple approaches 

First Approach 
just consider only "year" column and ignore day and month 

second approach 

Create new column for binary feature is_release_date_known
1 indicates a specific date or quarter is known.
0 indicates the entry is "to be announced" or "coming soon".
For the main release date feature (fractional_year) = year + (day of year - 1)/days in year
```py
quarter_map = {
    'Q1': 0.125,
    'Q2': 0.375,
    'Q3': 0.625,
    'Q4': 0.875
}

df['release_date'] = pd.to_datetime(df['release_date'])

df['fractional_year'] = df['release_date'].dt.year + (df['release_date'].dt.dayofyear - 1) / (365 + df['release_date'].dt.is_leap_year.astype(int))

df['fractional_year'] = df['release_date'].apply(lambda x: int(x[:4]) + quarter_map[x[5:]] if isinstance(x, str) and x.startswith('Q') else x)
```

Third Approach 
guide : https://medium.com/%40paghadalsneh/handling-date-and-time-data-in-machine-learning-a-comprehensive-guide-5d30141cbfec 

splitting date into 3 features (day,month,year)

Fourth Approach 
Cyclical encoding 



In [9]:
# Preprocessing for achievements total 
#df_infobase['achievements_total'].fillna("0",inplace=True)
df_infobase['achievements_total'] = pd.to_numeric(df_infobase['achievements_total'], errors='coerce').fillna(0).astype(int)
#df_infobase[[df_infobase['achievements_total'] == None]] = 0
df_infobase['achievements_total'].isnull().sum()
df_infobase['steam_achievements'] = (df_infobase['achievements_total'] != 0).astype(int)
# all nulls be zero 
#imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
#df_infobase['achievements_total'] = imputer.fit_transform(df_infobase['achievements_total'])

df_infobase

,appid,name,metacritic,steam_achievements,steam_trading_cards,workshop_support,genres,achievements_total,supported_platforms,is_release_date_known,year,fraction_sin,fraction_cos
0,2574000,Femboy Burgers,NaN,0,True,True,"Casual, Indie",0,"['windows', 'mac', 'linux']",1,2020,-0.999991,0.004304
1,2574120,PPA Pickleball Tour 2025,NaN,1,True,True,"Indie, Simulation, Sports",18,"['windows', 'mac', 'linux']",1,2020,0.008607,-0.999963
2,2573200,Squeaky Squad,NaN,1,True,True,"Action, Adventure, Indie",27,"['windows', 'mac', 'linux']",1,2020,0.858764,0.512371
3,2573440,Paradox Metal,NaN,0,True,True,"Action, Early Access",0,"['windows', 'mac', 'linux']",0,0,0.000000,1.000000
4,2569520,Naturpark Lillebælt VR,NaN,0,True,True,"Action, Adventure",0,"['windows', 'mac', 'linux']",1,2020,-0.871706,-0.490029
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99162,1548850,Six Days in Fallujah,NaN,1,False,False,"Action, Early Access",34,['windows'],1,2020,0.501242,-0.865307
99163,2478130,PROJECT SURVIVAL #Working title,NaN,0,False,False,"Action, Adventure, RPG",0,['windows'],0,0,0.000000,1.000000
99164,3272980,Siren's Well,NaN,0,False,False,"Action, Adventure",0,"['windows', 'linux']",0,0,0.000000,1.000000
99165,2054150,Tower Defender VR: Last Adventure,NaN,0,False,False,"Casual, Indie, RPG, Strategy",0,['windows'],1,2020,0.008607,-0.999963
